In [14]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
    <script> code_show=true; function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();} code_show = !code_show} $( document ).ready(code_toggle);</script>
    <form action="javascript:code_toggle()"><input type="submit" value="Hide/show all code."></form>
''') 

In [15]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

In [18]:
db, conn = pgconnect(credentials)

Connected successfully.


In [19]:
from sqlalchemy import inspect
inspect(db).get_schema_names()

['information_schema', 'nswfuel', 'prices', 'public', 'unitsofstudy']

In [24]:
inspect(db).get_table_names(schema='nswfuel')

['prices', 'observations', 'stations', 'companies', 'fuel']

In [23]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [25]:
conn.execute("set search_path to public")

In [28]:
conn.execute("select * from spatial_ref_sys")

In [30]:
query(conn, "select * from spatial_ref_sys")

,srid,auth_name,auth_srid,srtext,proj4text
0,3819,EPSG,3819,"GEOGCS[""HD1909"",DATUM[""Hungarian_Datum_1909"",S...","+proj=longlat +ellps=bessel +towgs84=595.48,12..."
1,3821,EPSG,3821,"GEOGCS[""TWD67"",DATUM[""Taiwan_Datum_1967"",SPHER...",+proj=longlat +ellps=aust_SA +no_defs
2,3824,EPSG,3824,"GEOGCS[""TWD97"",DATUM[""Taiwan_Datum_1997"",SPHER...","+proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,..."
3,3889,EPSG,3889,"GEOGCS[""IGRS"",DATUM[""Iraqi_Geospatial_Referenc...","+proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,..."
4,3906,EPSG,3906,"GEOGCS[""MGI 1901"",DATUM[""MGI_1901"",SPHEROID[""B...","+proj=longlat +ellps=bessel +towgs84=682,-203,..."
...,...,...,...,...,...
8495,103967,ESRI,103967,"PROJCS[""NAD_1983_HARN_Adj_WI_Vilas_Feet (depre...",+proj=lcc +lat_0=45.625 +lon_0=-89.48888888888...
8496,103968,ESRI,103968,"PROJCS[""NAD_1983_HARN_Adj_WI_Walworth_Feet (de...",+proj=lcc +lat_0=41.6694444444444 +lon_0=-88.5...
8497,103969,ESRI,103969,"PROJCS[""NAD_1983_HARN_Adj_WI_Washburn_Feet (de...",+proj=lcc +lat_0=44.2666666666667 +lon_0=-91.7...
8498,103970,ESRI,103970,"PROJCS[""NAD_1983_HARN_Adj_WI_Waushara_Feet (de...",+proj=lcc +lat_0=43.7083333333333 +lon_0=-89.2...
